# Blackjack AR

In [1]:
from __future__ import print_function       # Python 2/3 compatibility
import cv2                                  # Import the OpenCV library
import mediapipe as mp                      # Import Mediapipe
import numpy as np                          # Import Numpy library

from enum import Enum
from ast import arg
import sys

In [2]:
class BlackjackAction(Enum):
    NONE = 0
    HIT = 1
    STAND = 2
    DOUBLE = 3
    SPLIT = 4

class GestureRecognizer:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
    def detect_gesture(self, hand_landmarks):
        """
        Detecta el gesto de la mano y retorna la acción correspondiente de blackjack
        """
        total_fingers, fingers_up, is_right_hand = self.count_fingers_up(hand_landmarks)
        
        # Detectar gestos específicos para blackjack
        if self._is_hit_gesture(total_fingers, fingers_up):
            return BlackjackAction.HIT
        elif self._is_stand_gesture(total_fingers, fingers_up):
            return BlackjackAction.STAND
        elif self._is_double_gesture(total_fingers, fingers_up):
            return BlackjackAction.DOUBLE
        elif self._is_split_gesture(total_fingers, fingers_up):
            return BlackjackAction.SPLIT
        
        return BlackjackAction.NONE

    def _is_hit_gesture(self, total_fingers, fingers_up):
        """Un dedo levantado (índice) para pedir carta"""
        return total_fingers == 1 and fingers_up[1]

    def _is_stand_gesture(self, total_fingers, fingers_up):
        """Puño cerrado para plantarse"""
        return total_fingers == 0 or (total_fingers == 1 and fingers_up[0] == 1)

    def _is_double_gesture(self, total_fingers, fingers_up):
        """Dos dedos levantados (índice y medio) para doblar"""
        return total_fingers == 2 and fingers_up[1] and fingers_up[2]
    
    def _is_split_gesture(self, total_fingers, fingers_up):
        """Dos dedos levantados (pulgar e índice) para dividir las cartas"""
        return total_fingers == 2 and fingers_up[0] and fingers_up[1]

    def count_fingers_up(self, hand_landmarks):
        """
        Cuenta dedos levantados y determina su posición
        """
        finger_tips = [4, 8, 12, 16, 20]
        finger_bases = [2, 5, 9, 13, 17]
        fingers_up = []

        # Detectar mano derecha/izquierda
        is_right_hand = hand_landmarks.landmark[2].x < hand_landmarks.landmark[0].x

        # Comprobar pulgar
        thumb_tip_x = hand_landmarks.landmark[4].x
        thumb_ip_x = hand_landmarks.landmark[3].x
        fingers_up.append(thumb_tip_x < thumb_ip_x if is_right_hand else thumb_tip_x > thumb_ip_x)

        # Comprobar resto de dedos
        for tip, base in zip(finger_tips[1:], finger_bases[1:]):
            fingers_up.append(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[base].y)

        return sum(fingers_up), fingers_up, is_right_hand

    def draw_debug_info(self, frame, action, hand_landmarks, fingers_up, is_right_hand):
        """
        Dibuja información de depuración en el frame
        """
        # Dibujar landmarks de la mano
        self.mp_drawing.draw_landmarks(
            frame,
            hand_landmarks,
            self.mp_hands.HAND_CONNECTIONS,
            self.mp_drawing_styles.get_default_hand_landmarks_style(),
            self.mp_drawing_styles.get_default_hand_connections_style()
        )

        # Mostrar acción detectada
        action_text = f"Action: {action.name}"
        cv2.putText(frame, action_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Mostrar estado de los dedos
        finger_names = ["Thumb", "Index", "Middle", "Ring", "Pinky"]
        for i, (finger, is_up) in enumerate(zip(finger_names, fingers_up)):
            status = "Up" if is_up else "Down"
            cv2.putText(frame, f"{finger}: {status}", (10, 60 + (30 * i)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            
        cv2.putText(frame, f"Mano derecha: {is_right_hand}", (10, 210),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [3]:
import random
from enum import Enum
from typing import List, Tuple, Optional

class Card:
    def __init__(self, suit, value, image):
        self.suit = suit
        self.value = value
        self.image = image

class GameState(Enum):
    WAITING_FOR_BET = 0
    PLAYER_TURN = 1
    DEALER_TURN = 2
    GAME_OVER = 3

class BlackjackGame:
    def __init__(self):
        self.deck = []
        self.player_hands = [[]]  # Lista de manos para permitir splits
        self.dealer_hand = []
        self.current_hand_index = 0
        self.player_chips = 1000
        self.current_bet = 0
        self.game_state = GameState.WAITING_FOR_BET
        self.initialize_deck()

    def initialize_deck(self):
        """Inicializa el mazo de cartas"""
        suits = ['clubs', 'diamonds', 'hearts', 'spades']
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
        self.deck = [Card(suit, value, f'cartas/{value}_of_{suit}.png') for suit in suits for value in values]
        random.shuffle(self.deck)

    def get_game_state(self) -> dict:
        """Retorna el estado actual del juego"""
        return {
            'player_hands': self.player_hands,
            'dealer_hand': self.dealer_hand,
            'current_hand_index': self.current_hand_index,
            'player_chips': self.player_chips,
            'current_bet': self.current_bet,
            'game_state': self.game_state,
            'can_split': self.can_split() if self.game_state == GameState.PLAYER_TURN else False
        }

    def can_split(self) -> bool:
        """Verifica si el jugador puede dividir su mano"""
        if (self.game_state != GameState.PLAYER_TURN or 
            self.current_hand_index >= len(self.player_hands)):
            return False
            
        current_hand = self.player_hands[self.current_hand_index]
        return (len(current_hand) == 2 and 
                current_hand[0].value == current_hand[1].value and 
                len(self.player_hands) < 4 and 
                self.player_chips >= self.current_bet)

    def process_action(self, action: BlackjackAction) -> Tuple[bool, str]:
        """Procesa una acción del jugador y retorna (éxito, mensaje)"""
        if self.game_state != GameState.PLAYER_TURN:
            return False, "No es tu turno"

        if self.current_hand_index >= len(self.player_hands):
            self.dealer_turn()
            return False, "Turno del dealer"

        current_hand = self.player_hands[self.current_hand_index]

        if action == BlackjackAction.HIT:
            current_hand.append(self.deck.pop())
            if self.calculate_hand_value(current_hand) > 21:
                self.next_hand()
            return True, "Carta repartida"

        elif action == BlackjackAction.STAND:
            self.next_hand()
            return True, "Te plantas"

        elif action == BlackjackAction.DOUBLE:
            if len(current_hand) != 2 or self.player_chips < self.current_bet:
                return False, "No puedes doblar"
            self.player_chips -= self.current_bet
            self.current_bet *= 2
            current_hand.append(self.deck.pop())
            self.next_hand()
            return True, "Apuesta doblada"

        elif action == BlackjackAction.SPLIT:
            if not self.can_split():
                return False, "No puedes dividir"
            self.player_chips -= self.current_bet
            new_hand = [current_hand.pop()]
            new_hand.append(self.deck.pop())
            current_hand.append(self.deck.pop())
            self.player_hands.append(new_hand)
            return True, "Mano dividida"

        return False, "Acción no válida"

    def next_hand(self):
        """Pasa a la siguiente mano o al turno del dealer"""
        self.current_hand_index += 1
        if self.current_hand_index >= len(self.player_hands):
            self.dealer_turn()

    def dealer_turn(self):
        """Ejecuta el turno del dealer"""
        self.game_state = GameState.DEALER_TURN
        while self.calculate_hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.pop())
        self.end_game()

    def calculate_hand_value(self, hand: List[Card]) -> int:
        """Calcula el valor de una mano teniendo en cuenta los ases"""
        value = 0
        aces = 0
        
        for card in hand:
            if card.value in ['jack', 'queen', 'king']:
                value += 10
            elif card.value == 'ace':
                aces += 1
            else:
                value += int(card.value)
        
        for _ in range(aces):
            if value + 11 <= 21:
                value += 11
            else:
                value += 1
                
        return value

    def deal_initial_cards(self):
        """Reparte las cartas iniciales"""
        self.player_hands = [[self.deck.pop(), self.deck.pop()]]
        self.dealer_hand = [self.deck.pop(), self.deck.pop()]
        self.current_hand_index = 0
        self.game_state = GameState.PLAYER_TURN

    def place_bet(self, amount: int) -> bool:
        """Coloca una apuesta"""
        if self.game_state != GameState.WAITING_FOR_BET or amount > self.player_chips:
            return False
        self.current_bet = amount
        self.player_chips -= amount
        self.deal_initial_cards()
        return True

    def end_game(self):
        """Finaliza el juego y calcula las ganancias"""
        self.game_state = GameState.GAME_OVER
        dealer_value = self.calculate_hand_value(self.dealer_hand)
        dealer_bust = dealer_value > 21

        for hand in self.player_hands:
            player_value = self.calculate_hand_value(hand)
            if player_value > 21:
                continue  # Mano perdida
            elif dealer_bust or player_value > dealer_value:
                self.player_chips += self.current_bet * 2
            elif player_value == dealer_value:
                self.player_chips += self.current_bet  # Empate
        

In [4]:
desired_aruco_dictionary = "DICT_4X4_100"
 
# The different ArUco dictionaries built into the OpenCV library. 
ARUCO_DICT = {
  "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
  "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
  "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
  "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
  "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
  "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
  "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
  "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
  "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
  "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
  "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
  "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
  "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
  "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
  "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
  "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
  "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL
}

def showArucoDetections(corners, ids, frame):
    # Check that at least one ArUco marker was detected
    if len(corners) > 0:
      # Flatten the ArUco IDs list
      ids = ids.flatten()
       
      # Loop over the detected ArUco corners
      for (marker_corner, marker_id) in zip(corners, ids):
       
        # Extract the marker corners
        corners = marker_corner.reshape((4, 2))
        (top_left, top_right, bottom_right, bottom_left) = corners
         
        # Convert the (x,y) coordinate pairs to integers
        top_right = (int(top_right[0]), int(top_right[1]))
        bottom_right = (int(bottom_right[0]), int(bottom_right[1]))
        bottom_left = (int(bottom_left[0]), int(bottom_left[1]))
        top_left = (int(top_left[0]), int(top_left[1]))
         
        # Draw the bounding box of the ArUco detection
        cv2.line(frame, top_left, top_right, (0, 255, 0), 2)
        cv2.line(frame, top_right, bottom_right, (0, 255, 0), 2)
        cv2.line(frame, bottom_right, bottom_left, (0, 255, 0), 2)
        cv2.line(frame, bottom_left, top_left, (0, 255, 0), 2)
         
        # Calculate and draw the center of the ArUco marker
        center_x = int((top_left[0] + bottom_right[0]) / 2.0)
        center_y = int((top_left[1] + bottom_right[1]) / 2.0)
        cv2.circle(frame, (center_x, center_y), 4, (0, 0, 255), -1)
         
        # Draw the ArUco marker ID on the video frame
        # The ID is always located at the top_left of the ArUco marker
        cv2.putText(frame, "ARUCO: " + str(marker_id) + "Starting game", 
          (top_left[0], top_left[1] - 15),
          cv2.FONT_HERSHEY_SIMPLEX,
          0.5, (0, 255, 0), 2)

In [5]:
def print_game_state(game):
    """Imprime el estado actual del juego"""
    state = game.get_game_state()
    
    print("\n" + "="*50)
    print("ESTADO DEL JUEGO:")
    print(f"Fichas: {state['player_chips']} | Apuesta actual: {state['current_bet']}")
    
    # Mostrar cartas del dealer
    print("\nCartas del Dealer:")
    if state['game_state'] == GameState.PLAYER_TURN:
        # Durante el turno del jugador, solo mostrar la primera carta del dealer
        print(f"Carta visible: {state['dealer_hand'][0].value} de {state['dealer_hand'][0].suit}")
        print("Carta oculta: ???")
    else:
        dealer_cards = [f"{card.value} de {card.suit}" for card in state['dealer_hand']]
        print(f"Cartas: {', '.join(dealer_cards)}")
        print(f"Valor total: {game.calculate_hand_value(state['dealer_hand'])}")
    
    # Mostrar cartas del jugador
    print("\nTus manos:")
    for i, hand in enumerate(state['player_hands']):
        cards = [f"{card.value} de {card.suit}" for card in hand]
        value = game.calculate_hand_value(hand)
        current = " (Mano actual)" if i == state['current_hand_index'] and state['game_state'] == GameState.PLAYER_TURN else ""
        print(f"Mano {i+1}{current}: {', '.join(cards)} (Valor: {value})")
    
    # Mostrar acciones disponibles
    if state['game_state'] == GameState.PLAYER_TURN:
        print("\nAcciones disponibles:")
        print("- Pedir carta (índice arriba)")
        print("- Plantarse (puño cerrado)")
        if len(state['player_hands'][state['current_hand_index']]) == 2:
            print("- Doblar (índice y medio arriba)")
        if state['can_split']:
            print("- Dividir (pulgar e índice arriba)")
    
    print("="*50 + "\n")

In [6]:
def main():
  """
  Main method of the program.
  """
  # Check that we have a valid ArUco marker
  if ARUCO_DICT.get(desired_aruco_dictionary, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(
      args["type"]))
    sys.exit(0)
     
  # Load the ArUco dictionary
  print("[INFO] detecting '{}' markers...".format(
    desired_aruco_dictionary))
  

  this_aruco_dictionary = cv2.aruco.Dictionary_get(ARUCO_DICT[desired_aruco_dictionary])
  this_aruco_parameters = cv2.aruco.DetectorParameters_create()
   
  # Start the video stream
  cap = cv2.VideoCapture(0)
  gesture_recognizer = GestureRecognizer()
  game = BlackjackGame()
   
  with gesture_recognizer.mp_hands.Hands(
          model_complexity=0,
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5) as hands:
          
          while True:
              # Capturar frame
              ret, frame = cap.read()
              if not ret:
                  break
                  
              # Convertir el frame a RGB para MediaPipe
              frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              
              # Procesar el frame para detectar manos
              results = hands.process(frame_rgb)
              
              # Detect ArUco markers in the video frame
              (corners, ids, rejected) = cv2.aruco.detectMarkers(frame, this_aruco_dictionary, parameters=this_aruco_parameters)
              
              # Mostrar detecciones de ArUco
              showArucoDetections(corners, ids, frame)
              
              # Dibujar las detecciones de manos
              if results.multi_hand_landmarks:
                
                for hand_landmarks in results.multi_hand_landmarks:
                  gesture_recognizer.mp_drawing.draw_landmarks(
                      frame,
                      hand_landmarks,
                      gesture_recognizer.mp_hands.HAND_CONNECTIONS,
                      gesture_recognizer.mp_drawing_styles.get_default_hand_landmarks_style(),
                      gesture_recognizer.mp_drawing_styles.get_default_hand_connections_style())
                    
                  # Detectar gestos y acciones
                  total_fingers, fingers_up, is_right_hand = gesture_recognizer.count_fingers_up(hand_landmarks)
                  action = gesture_recognizer.detect_gesture(hand_landmarks)
                
                  # Dibujar información de depuración
                  gesture_recognizer.draw_debug_info(frame, action, hand_landmarks, 
                                                fingers_up, is_right_hand)
                  
              # Mostrar el frame
              cv2.imshow('ArUco + Hand Detection', frame)
              
              # Salir con 'q'
              if cv2.waitKey(1) & 0xFF == ord('q'):
                  break
          
          # Liberar recursos
          cap.release()
          cv2.destroyAllWindows()
   
if __name__ == '__main__':
  print(__doc__)
  main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_4X4_100' markers...


In [6]:
def main():
    """
    Main method of the program.
    """
    # Inicialización de ArUco y otros componentes existentes
    if ARUCO_DICT.get(desired_aruco_dictionary, None) is None:
        print("[INFO] ArUCo tag of '{}' is not supported".format(
            args["type"]))
        sys.exit(0)
    
    print("[INFO] detecting '{}' markers...".format(desired_aruco_dictionary))
    
    this_aruco_dictionary = cv2.aruco.Dictionary_get(ARUCO_DICT[desired_aruco_dictionary])
    this_aruco_parameters = cv2.aruco.DetectorParameters_create()
    
    # Inicializar el juego
    game = BlackjackGame()
    gesture_recognizer = GestureRecognizer()
    last_action = None
    action_cooldown = 0
    
    print("\n¡Bienvenido al Blackjack!")
    print(f"Tienes {game.player_chips} fichas.")
    success = game.place_bet(100)  # Apuesta inicial fija de 100
    if not success:
        print("Error al colocar la apuesta inicial")
        return
    print("Apuesta inicial de 100 fichas colocada.")
    
    cap = cv2.VideoCapture(0)
    
    with gesture_recognizer.mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        
        print_game_state(game)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            
            # Detect ArUco markers
            (corners, ids, rejected) = cv2.aruco.detectMarkers(
                frame, this_aruco_dictionary, parameters=this_aruco_parameters)
            
            # Mostrar detecciones de ArUco
            showArucoDetections(corners, ids, frame)
            
            current_action = BlackjackAction.NONE
            
            if results.multi_hand_landmarks and game.game_state == GameState.PLAYER_TURN:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Detectar gestos y acciones
                    total_fingers, fingers_up, is_right_hand = gesture_recognizer.count_fingers_up(hand_landmarks)
                    current_action = gesture_recognizer.detect_gesture(hand_landmarks)
                    
                    # Dibujar información de depuración
                    gesture_recognizer.draw_debug_info(frame, current_action, hand_landmarks, 
                                                     fingers_up, is_right_hand)
                    
                    # Procesar acción si es diferente a la última y no hay cooldown
                    if (current_action != BlackjackAction.NONE and 
                        current_action != last_action and 
                        action_cooldown == 0):
                        
                        success, message = game.process_action(current_action)
                        if success:
                            print(f"\n{message}")
                            print_game_state(game)
                            last_action = current_action
                            action_cooldown = 15  # Esperar 15 frames antes de aceptar otra acción
            
            # Reducir el cooldown
            if action_cooldown > 0:
                action_cooldown -= 1
            
            # Si el juego ha terminado, mostrar resultados y reiniciar
            if game.game_state == GameState.GAME_OVER:
                print("\n¡Juego terminado!")
                print(f"Tus fichas: {game.player_chips}")
                
                # Preguntar si quiere jugar otra mano
                if game.player_chips >= 100:  # Asegurar que tiene suficientes fichas
                    game = BlackjackGame()
                    game.player_chips = game.player_chips  # Mantener las fichas ganadas
                    game.place_bet(100)
                    print("\nNueva mano comenzada con apuesta de 100 fichas.")
                    print_game_state(game)
                else:
                    print("\n¡Te has quedado sin fichas! Fin del juego.")
                    break
            
            cv2.imshow('Blackjack AR', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    print(__doc__)
    main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_4X4_100' markers...

¡Bienvenido al Blackjack!
Tienes 1000 fichas.
Apuesta inicial de 100 fichas colocada.

ESTADO DEL JUEGO:
Fichas: 900 | Apuesta actual: 100

Cartas del Dealer:
Carta visible: 7 de diamonds
Carta oculta: ???

Tus manos:
Mano 1 (Mano actual): 9 de hearts, king de spades (Valor: 19)

Acciones disponibles:
- Pedir carta (índice arriba)
- Plantarse (puño cerrado)
- Doblar (índice y medio arriba)


Carta repartida

ESTADO DEL JUEGO:
Fichas: 900 | Apuesta actual: 100

Cartas del Dealer:
Cartas: 7 de diamonds, jack de hearts
Valor total: 17

Tus manos:
Mano 1: 9 de hearts, king de spades, 4 de diamonds (Valor: 23)


¡Juego terminado!
Tus fichas: 900

Nueva mano comenzada con apuesta de 100 fichas.

ESTADO DEL JUEGO:
Fichas: 900 | Apuesta actual: 100

Cartas del Dealer:
Carta visible: 3 de spades
Carta oculta: ???

Tus manos:
Mano 1 (Mano actual): 7 de hearts, 8 de clubs (Valor: 15)

Acci